## Gait Video Study 
### Identifying frames with HSRs in each video for each cohort and trial to establish break points and also evaluate the corresponding HSR labelling via the ground truth available. Further, downsample with smoothing to define fixed shape of the input tensor for models. 
#### Remember to preserve the original count of frames in a single stride (before down sampling via smoothing) for each stride to add as an additional artificial feature later to add information about speed of the subject to the model

In [1]:
import numpy as np
import cv2
import os
import glob
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
import scipy
from scipy import signal
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [2]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data'
frame_path_merged = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\multi_view_merged_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

### Utility functions 

In [5]:
#Saving the HSRframes.txt file to the hip_height_normalized\\ containing the final .csvs for analysis
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            print (glob.glob(path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'))
            try:
                if (not os.path.exists(merged_path+'\\'+video+'\\HSRframes.txt')):
                    HSR_frames_file = path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'+'\\HSRframes.txt'
                    shutil.copy(HSR_frames_file, merged_path+'\\'+video+'\\hip_height_normalized\\') 
                    print ('HSR for', video, 'copied')
                else:
                    print ('HSR for', video, 'exists')
            except Exception as e:
                print (e)

[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_214_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Proje

In [3]:
labels = ['frame_number'] + [o + '-'+ y for o in order for y in ['x', 'y', 'z']]

for cohort in cohorts[1:2]:
    for trial in trials[:1]:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos[2:3]:
            #Reading the ground truth HSR frames numbers
            trueHSR = open(merged_path+'\\'+video+'\\hip_height_normalized\\'+'\\HSRframes.txt').read()
            #Making a list containing ground truth HSRs out of the read file
            trueHSR_list = [int(a) for a in trueHSR.split(',')]
            print (trueHSR_list)
            
            frames = glob.glob(merged_path+'\\'+video+'\\hip_height_normalized\\*.csv')
            #First, we need to sort the frames since we need frames to appear in order to detect HSRs
            sorted_frames = sorted(frames,  key=lambda name: int(name.split('\\')[-1][:-4]))
            #Dataframe to hold all 12*3 features (right hip-x, right hip-y, ...) as columns and all frames as rows 
            #for each video 
            video_features = pd.DataFrame(columns = labels)
            for frame in sorted_frames:
                #Append the frame number also in the temp dataframe, since it the true HSR is given in frame number
                frame_no = int(frame.split('\\')[-1][:-4])
                frame_csv = pd.read_csv(frame, index_col = 0)
                #Appending the 36 features and frame number of each frame as a row for each video's dataframe 
                video_features.loc[len(video_features)] = np.append(frame_no, frame_csv[['x', 'y', 'z']].values.flatten())
            video_features = video_features.astype({'frame_number': 'int'})
            #Setting the frame number as the index 
            video_features.set_index('frame_number', inplace = True)
            display(video_features)
            

[35, 71, 107, 143, 179, 214, 250, 287, 320, 356, 392, 427, 462, 500, 536, 570, 606, 641, 674, 706, 737, 769, 800, 829, 860, 889, 917, 947, 977]


,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,39.449136,162.131598,100.184257,36.522369,143.974399,87.395076,34.058394,104.701142,26.561741,20.414385,...,26.447676,34.734083,148.392273,22.461395,38.667728,143.412639,21.829892,32.019455,139.504274,28.874307
1,39.453248,162.145605,100.192216,36.525768,143.985215,87.403087,34.926049,104.691449,26.578776,19.565527,...,26.439908,34.495741,116.735102,18.592852,38.969747,115.313934,19.548701,33.300731,108.728464,23.897386
2,39.777655,164.810217,99.938657,36.855551,146.956253,86.765595,35.172231,105.483416,26.752898,20.601070,...,25.791256,31.941744,116.303816,19.569412,37.405932,116.174572,19.671773,34.452235,109.563609,24.045646
3,40.576205,164.318651,99.697618,36.720703,144.624022,87.810100,35.076347,105.204018,26.674837,21.426205,...,23.986813,32.810106,117.280771,18.661366,38.263885,117.211643,18.723840,34.406710,110.549174,23.129102
4,40.814521,165.856493,99.703791,37.859332,146.285743,87.614330,35.283373,105.706628,26.815713,20.673328,...,26.676634,34.783622,117.879816,18.743699,39.390810,117.722684,18.861933,34.491380,109.765016,24.122696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,33.257117,130.605967,100.000000,30.563866,147.224373,88.762443,31.441949,155.152298,13.520264,7.069172,...,41.828280,31.471683,175.545710,0.000083,35.720899,172.229849,2.211375,30.152580,159.485252,10.643699
985,33.964744,130.588627,100.000000,30.616590,148.772599,85.665044,29.207571,153.064447,12.826504,7.072159,...,39.712779,29.902723,172.414484,0.000083,34.193547,170.138093,1.522363,28.597740,157.359355,9.977237
986,34.645799,129.410401,100.000000,29.900080,148.791184,85.624949,29.172002,153.048535,12.811359,6.333335,...,11.047440,29.883017,172.375801,0.000083,34.202578,170.098442,1.523092,27.835152,156.350980,10.617848


In [4]:
#Function to plot the given feature across frames of a video and the corresponding ground truth HSRs
def plot_true_HSR(signal, index, HSRindices, video, name):
    fig= plt.figure(figsize = (8.5, 3))
    ax1 = fig.add_subplot(111)
    ax1.plot(index, signal, color = 'b', ls='solid', marker='*', \
            markerfacecolor = 'r', markeredgecolor = 'r', ms = 10, markevery= HSRindices, label = 'true HSR')
    plt.xticks(index[0::40], fontsize = 8)
    plt.legend()
    plt.title(name)
    plt.show()
    plt.savefig('HSR_detection_figs\\'+ video+ '_' + name + '.png', dpi = 250)
#     append = np.append(ax.get_xticks(), trueHSR_list)
#     # ax.set_xticks(trueHSR_list)
#     # ax.set_xticks(temp.index)
#     plt.show()

In [20]:
#Do we need the missing frame information 
#We might need to annotate the ground truth HSR frames in the plot 
#https://stackoverflow.com/questions/14432557/matplotlib-scatter-plot-with-different-text-at-each-data-point
#For any algorithm to find the HSR's, we might need to fill missing values in the series we are using to detect HSR
#The ground truth HSR markers are not shown when the values are NAN- solve?

#If some frames are missing, appending those rows with NaN values 
video_features_complete = video_features.reindex(range(video_features.index[-1]))
#Indices for the ground truth HSR in the video_features dataframe (not containing placeholders for 
#the missing frames)
trueHSR_indices = [i for i, val in enumerate(video_features.index) if val in trueHSR_list] 
#Indices for the ground truth HSR in the video_features_complete dataframe (containing NaN valued placeholders for 
#the missing frames)
trueHSR_indices_complete = [i for i, val in enumerate(video_features_complete.index) if val in trueHSR_list] 

#Right Heel Height 
#Plotting the right heel height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right heel-z']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right heel-z')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(video_features['right heel-z'].values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right heel-z')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right heel-z']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right heel-z')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(video_features_complete['right heel-z'].values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right heel-z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
#Right Toe 1 - Right heel height 

#Plotting the right toe height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right toe 1-z']-video_features['right heel-z']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right toe 1-z - right heel-z')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(series.values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right toe 1-z - right heel-z')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right toe 1-z'] - video_features_complete['right heel-z']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right toe 1-z - right heel-z')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(series_complete.values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right toe 1-z - right heel-z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#Right Toe 2 - Right heel height 

#Plotting the right toe height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right toe 2-z']-video_features['right heel-z']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right toe 2-z - right heel-z')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(series.values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right toe 2-z - right heel-z')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right toe 2-z'] - video_features_complete['right heel-z']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right toe 2-z - right heel-z')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(series_complete.values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right toe 2-z - right heel-z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
#Right Toe 2 - Right heel y coordinate

#Plotting the right toe height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right toe 2-y']-video_features['right heel-y']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right toe 2-y - right heel-y')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(series.values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right toe 2-y- right heel-y')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right toe 2-y'] - video_features_complete['right heel-y']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right toe 2-y - right heel-y')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(series_complete.values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right toe 2-y - right heel-y')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
#Right Toe 1 height 

#Plotting the right toe height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right toe 1-z']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right toe 1-z')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(video_features['right toe 1-z'].values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right toe 1-z')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right toe 1-z']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right toe 1-z')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(video_features_complete['right toe 1-z'].values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right toe 1-z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
#Right Toe 2 height 

#Plotting the right toe height (not containing placeholders for the missing frames) and ground truth HSR 
series = video_features['right toe 2-z']
plot_true_HSR(series.values, series.index, trueHSR_indices, video, 'right toe 2-z')

#Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
filtered_values = signal.savgol_filter(video_features['right toe 2-z'].values, window_length = 21, polyorder=3)
plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered right toe 2-z')

#Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
series_complete = video_features_complete['right toe 2-z']
plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
              'complete right toe 2-z')

#Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
filtered_values_complete = signal.savgol_filter(video_features_complete['right toe 2-z'].values, \
                                                window_length = 5, polyorder=3)
plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
              'filtered complete right toe 2-z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
video_features.iloc[trueHSR_indices]

,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
35,44.517980,172.014059,99.705280,41.568783,155.046067,85.403315,36.929948,109.076727,26.467598,23.067544,...,19.813427,36.548121,124.197186,16.441984,42.050846,121.366232,18.335622,35.182741,111.928101,24.571883
71,42.376519,163.574809,100.211016,40.508848,148.442553,85.303072,35.989472,105.214995,26.677163,19.674789,...,18.580256,36.549509,119.930886,16.922642,41.934377,118.420914,17.940145,35.295146,110.543436,23.142435
107,42.699930,169.286289,99.798314,37.465127,161.756344,79.749637,34.497982,135.163208,9.784134,12.561892,...,13.126988,35.917547,149.927564,0.000104,41.344060,148.446085,0.998845,31.891467,139.264842,7.056909
179,37.544542,128.741122,100.000000,35.311477,136.961333,99.701596,43.347224,161.343132,13.572796,7.010362,...,22.544503,41.192643,181.805727,0.000083,46.992030,180.553650,0.847874,42.062269,165.648359,10.714627
214,36.262061,133.467988,100.000000,37.184827,148.629724,95.772923,41.030107,157.087381,20.188602,2.911525,...,9.951701,46.183882,178.430321,6.054548,50.390214,174.028852,8.985438,39.055041,163.562040,15.890054
250,37.422431,145.169757,100.000000,36.469129,147.878068,93.697088,41.856734,160.267966,15.510506,7.604175,...,10.207589,50.021194,183.700822,0.000083,52.781234,179.354507,2.889994,38.437051,166.790538,11.175784
356,34.107476,134.103897,100.000000,29.790273,145.662257,76.226777,27.292185,141.293126,9.126904,6.809035,...,12.197873,33.656408,155.088937,0.000083,34.984603,151.876166,2.134115,25.394297,148.876403,4.093608
392,34.861089,129.619940,100.232741,31.676461,132.385819,78.268495,25.401816,126.027608,12.041776,9.894115,...,18.221088,31.215103,144.217553,0.000084,34.746759,139.867333,2.894938,21.180333,130.461328,9.089458
427,32.896007,143.294372,99.804554,30.234537,141.882674,80.268439,23.736036,131.524245,15.492275,11.194164,...,12.372689,24.451366,151.266684,2.405764,28.078859,147.897349,4.650611,21.582755,133.793655,13.981147


In [16]:
video_features

,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,39.449136,162.131598,100.184257,36.522369,143.974399,87.395076,34.058394,104.701142,26.561741,20.414385,...,26.447676,34.734083,148.392273,22.461395,38.667728,143.412639,21.829892,32.019455,139.504274,28.874307
1,39.453248,162.145605,100.192216,36.525768,143.985215,87.403087,34.926049,104.691449,26.578776,19.565527,...,26.439908,34.495741,116.735102,18.592852,38.969747,115.313934,19.548701,33.300731,108.728464,23.897386
2,39.777655,164.810217,99.938657,36.855551,146.956253,86.765595,35.172231,105.483416,26.752898,20.601070,...,25.791256,31.941744,116.303816,19.569412,37.405932,116.174572,19.671773,34.452235,109.563609,24.045646
3,40.576205,164.318651,99.697618,36.720703,144.624022,87.810100,35.076347,105.204018,26.674837,21.426205,...,23.986813,32.810106,117.280771,18.661366,38.263885,117.211643,18.723840,34.406710,110.549174,23.129102
4,40.814521,165.856493,99.703791,37.859332,146.285743,87.614330,35.283373,105.706628,26.815713,20.673328,...,26.676634,34.783622,117.879816,18.743699,39.390810,117.722684,18.861933,34.491380,109.765016,24.122696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,33.257117,130.605967,100.000000,30.563866,147.224373,88.762443,31.441949,155.152298,13.520264,7.069172,...,41.828280,31.471683,175.545710,0.000083,35.720899,172.229849,2.211375,30.152580,159.485252,10.643699
985,33.964744,130.588627,100.000000,30.616590,148.772599,85.665044,29.207571,153.064447,12.826504,7.072159,...,39.712779,29.902723,172.414484,0.000083,34.193547,170.138093,1.522363,28.597740,157.359355,9.977237
986,34.645799,129.410401,100.000000,29.900080,148.791184,85.624949,29.172002,153.048535,12.811359,6.333335,...,11.047440,29.883017,172.375801,0.000083,34.202578,170.098442,1.523092,27.835152,156.350980,10.617848


### Downsample with smoothing to define fixed shape input tensor for models

In [ ]:
#Use mean with disjoint windows to downsample while smoothing 
#Make sure to preserve count of frames in a frame before smoothing to add as a feature 

